In [ ]:
import numpy as np, matplotlib.pyplot as plt
from collections import OrderedDict as od
from scipy.special import comb as C
from scipy.stats import linregress
from statistics import median
from scipy.optimize import fminbound
import json
import itertools
from tqdm.notebook import tqdm
#from os import walk
#import re

import mpmath as mp
mp.mp.dps = 50
mp.mp.pretty = True
print(mp.mp)

from math import sqrt
from joblib import Parallel, delayed

In [ ]:
def get_entropy_from_culling_factor_v2(c):
    S = od()
    sn = 0
    for E in c:
        S[E] = np.log(c[E])
        S[E] += sn
        sn += np.log(1 - c[E])
    return S

def get_entropy_from_culling_factor_with_heat_v2(c):
    S = od()
    sn = 0
    for E in c:
        S[E] = np.log(c[E])
        S[E] += sn
        sn += np.log(1 - c[E])
    return S

In [ ]:
def read_files_v2(parameters):
    q=parameters["q"];
    L=parameters["L"];
    R=parameters["R"];
    D=parameters["D"];
    nSteps=parameters["nSteps"];
    seed=parameters["seed"];
    heat=parameters["heat"];
    N = L * L;
    #name = "./datasets/2DBlume{}_q{}_D{}_N{}_R{}_nSteps{}_run{}X.txt".format("Heating" * heat, q, "{0:07.6f}".format(D), N, R, nSteps, seed)
    name = "./datasets/2DBlumeOld(FloatEnergy)/2DBlume{}_q{}_D{}_N{}_R{}_nSteps{}_run{}X.txt".format("Heating" * heat, q, "{0:07.6f}".format(D), N, R, nSteps, seed)
    #print(name)
    culling_factor = od()
    #print(N, R, nSteps, heat)
    with open(name) as f:
        for line in f:
            E, c = map(float, line.split())
            culling_factor[E] = c
    parameters["culling_factor"] = culling_factor
    if not heat:
        parameters["S"] = get_entropy_from_culling_factor_v2(culling_factor)
    else:
        parameters["S"] = get_entropy_from_culling_factor_with_heat_v2(culling_factor)
    
    return parameters

def read_tuple_files(tpl):
    if tpl[1]["heat"]:
        return {
            "cool": read_files_v2(tpl[0]),
            "heat": read_files_v2(tpl[1])
        }

def stitch_S(heat, cool):
    #not infinite values
    niv = sorted( list({E for E, S in heat["S"].items() if not np.isinf(S)} &
                       {E for E, S in cool["S"].items() if not np.isinf(S)}) )
    #chosen not infinite values
    cniv = niv[len(niv) // 3 : 2 * len(niv) // 3]
    #print(cniv)
    shift = 0
    count = 0
    for E in cniv:
        shift += heat["S"][E] - cool["S"][E]
        count += 1
    shift /= count

    result = od()
    for E in cool["S"]:
        if E < median(cniv):
            result[E] = cool["S"][E]
    for E in heat["S"]:
        if E >= median(cniv):
            result[E] = heat["S"][E] - shift
    return {"S": result, "L": heat["L"], "D": heat["D"], "R": heat["R"]}

def calc_hc(st, T):

    fe = mp.mpf(1.0) * np.array([e for e, S in st['S'].items()])
    fS = mp.mpf(1.0) * np.array([S for e, S in st['S'].items()])

    exp = np.frompyfunc(mp.exp, 1, 1)
    w = exp(fS - fe / T)

    Z = w.sum()
    E = (w * fe).sum()
    E_sq = (w * fe * fe).sum()
    E_qr = (w * fe * fe * fe * fe).sum()
    avgE = E / Z;
    avgE_sq = E_sq / Z;
    avgE_qr = E_qr / Z;
    # T, C, average E, BinderCumulant
    return T, float( (avgE_sq - avgE * avgE) / (T * T) ), float( avgE ), float(1 - (avgE_qr / (3 * avgE_sq * avgE_sq)))


In [ ]:
T_crit = {}

In [ ]:
D = 1.967
L = [x for x in [8, 12, 16, 20, 24, 32, 48, 64, 80, 96] if x >= 0]


R = [32768, 131072]

cool, heat = [{
    "q": 3,
    "D": D,
    "L": l,
    #"R" : R,
    "nSteps" : 10,
    "seed" : 0,
    "heat": False
} for l in L], [{
    "q": 3,
    "D": D,
    "L": l,
    #"R" : R,
    "nSteps" : 10,
    "seed" : 0,
    "heat": True
} for l in L]
print('D={}'.format(D))

parameters = []
for c, h in zip(cool, heat):
    try:
        c['R'] = R[1]
        h['R'] = R[1]
        parameters.append(read_tuple_files((c, h)))
    except:
        try:
            c['R'] = R[0]
            h['R'] = R[0]
            parameters.append(read_tuple_files((c, h)))
        except:
            pass
        
for i, x in enumerate(tqdm(parameters)):
    parameters[i]["stitched"] = stitch_S(x["heat"], x["cool"])
print('L in {}'.format(", ".join([str(x["stitched"]["L"]) for x in parameters]) ))


In [ ]:
D

In [ ]:
BC_crit = {} #{"T1": , "BC1": }

In [ ]:
0.4 + 1.8/10.15

In [ ]:
for p in tqdm(parameters):
    st = p["stitched"]
    L = st["L"]
    if L < 20:
        continue
    BC_crit[L] = {}
    a = 0.57
    b = 0.65
    #T_list = np.linspace(0.3, 4, 100)
    #for T in tqdm(T_list):
        # T, C, average E, BinderCumulant
    t, bc, _, _ = fminbound(lambda T: calc_hc(st, T)[3], a, b, maxfun=500, full_output=1, disp=False)
        #t, c, _, bc = calc_hc(st, T)
    BC_crit[L]["T"] = float(t)
    BC_crit[L]["BC"]= float(bc)
    print(BC_crit[L])


In [ ]:
with open("./old_tmp/BC_crit_D{}_second_peak.txt".format(D), 'w') as fp:
    json.dump(BC_crit, fp)
    #for L in T_crit:
    #    fp.write('{}\t{}\t{}\n'.format(L, T_crit[L]['T_crit'], T_crit[L]['BC_crit']))
with open('./old_tmp/BC_crit_D{}_second_peak.tsv'.format(D), 'w') as f:
    f.write('L\tT_crit\tBC_crit\n');
    for L in BC_crit:
        f.write('\t'.join([str(L), str(BC_crit[L]['T']), str(BC_crit[L]['BC'])]) + '\n')


In [ ]:
with open("./old_tmp/BC_crit_D{}_second_peak.txt".format(D), 'r') as fp:
    BC_crit = json.load(fp)


In [ ]:
del BC_crit[12]
BC_crit

In [ ]:
for D in [1.0, 1.5, 1.6, 1.7, 1.75, 1.87, 1.9, 1.92, 1.94, 1.95, 1.96, 1.962, 1.964, 1.965, 1.965, 1.966, 1.967, 1.97, 1.98, 1.99]:
    try:
        with open('./old_tmp/BC_crit_D{}_second_peak.txt'.format(D), 'r') as f:
            BC_crit = json.load(f)

        with open('./old_tmp/BC_crit_D{}_second_peak.tsv'.format(D), 'w') as f:
            f.write('L\tT_crit\tBC_crit\n');
            for L in BC_crit:
                f.write('\t'.join([L, str(BC_crit[L]['T']), str(BC_crit[L]['BC'])]) + '\n')
    except:
        print(D)